In [ ]:
import random
import time
import os

In [ ]:
import numpy as np
from IPython.display import clear_output
from io import BytesIO
import matplotlib.pyplot as plt
import pandas as pd
import requests
from PIL import Image
from copy import deepcopy
from time import sleep

In [ ]:
def img_load(url):
        "Take in an url and return a printable image"
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        return img
JP = img_load("https://www.linkpicture.com/q/JackPot-min.png")


In [ ]:
Loaded_pics = {}
Loaded_pics["JackPot"] = JP

In [ ]:
#@title Special Mod for Inventory Mod {display-mode: "form"}
def place_holder(n):
    return("{:"+str(n)+"}")

def sandwhich(n,text,thickness,syms = "*"):
    return("{}".format(syms*thickness)+place_holder(n).format(text)+"{}".format(syms*thickness))

def box_print(text,thickness=1,syms_h ="~",syms_v ="|",to_print = True,n_unicode = 0):
    "Print Text surronded by boxes"
    max_len = 0
    for i in text.split("\n"):
        if len(i) > max_len:
            max_len = len(i)
    box_length = round(max_len + 2*thickness + n_unicode*1.5)
    string = syms_h*box_length
    for line in text.split("\n"):
        string += "\n"+sandwhich(max_len,line,thickness,syms_v)
    string += "\n"+syms_h*box_length
    if to_print:
        print(string)
        return
    return string

# Slot Machine Mod

In [ ]:
#@title SLOT Machine  Mod {display-mode: "form"}
def slot_machine_sim(num_draws):
    "Simulate the Slot Machine,record if Lose or Win"
    Winning_COMBINATION = get_Winning_COMBINATION()
    RESULT = []
    for i in range(num_draws):
        result = [test.spinWheel(),test.spinWheel(),test.spinWheel()]
        RESULT.append(result in Winning_COMBINATION)
    return RESULT
     

def get_Combination(ITEMS =  ["🍑", "🍒", "🍇", "🔔", "👑", "🎯"],repeat = True):
    "Get a 3 place combination for all"
    COMBINATION = []
    for place_1 in ITEMS:
        com = [1,2,3]
        com[0] = place_1
        L2 = deepcopy(ITEMS)
        if not repeat:
            L2.remove(place_1)
        for place_2 in L2:
            com[1] = place_2
            L3 = deepcopy(L2)
            if not repeat:
                L3.remove(place_2)
            for place_3 in L3:
                com[2] = place_3
                COMBINATION.append(deepcopy(com))
    return COMBINATION

def is_strike(list,n):
    "Return True if the first n objects are the same"
    result = True
    for i in range(n-1):
        if list[i]!=list[i+1]:
            result = False
    return result

def get_Winning_COMBINATION():
    "Get a list of winning combination"
    Winning_COMBINATION = []
    for comb in get_Combination():
        if is_strike(comb,3):
            Winning_COMBINATION.append(comb)
        else: #If not strike
            if (is_strike(comb,2) and comb[2] == "👑" and comb[0]!= "🎯"):
                Winning_COMBINATION.append(comb)
            elif (comb[0]=="🍑"):
                Winning_COMBINATION.append(comb)
    return Winning_COMBINATION

class slot_machine():
    def __init__(self,INIT_BALANCE = 1200):
        #Constants:
        self.COMBINATION = ["👑👑👑","🔔🔔🔔/👑","🍇🍇🍇/👑","🍒🍒🍒/👑","🍑🍑🍑","🍑🍑⭕","🍑⭕⭕"]
        self.PROBABILITY = [(1/24)**3,(3/24)**2*(4/24),(6/24)**2*(7/24),(6/24)**2*(7/24),(6/24)**3,(6/24)**2,(1/6)**1]
        self.PRIZES = [250,20,14,10,7,5,2]
        self.firstWheel = None
        self.secondWheel = None
        self.thirdWheel = None
        self.balance = INIT_BALANCE
        self.ticket_price = self.get_prize(Jackpot = INIT_BALANCE)
        self.Rules = ('''Welcome to the Slot Machine 
                While your Balance is Positive, You'll be asked if you want to play.
                Every turn, you can choose how many 🎫 tickets to purchase, and different combination 
                will have different pay outs. If you get JACKPOT 🎯🎯🎯, you will get 
                everything in the slot machine! Good Luck!
                The Payoff Rates are the following...
                ''')
    def shine(self,t=2.5,t_b = 0.05):
        "shine for t seconds"
        n = t/t_b
        for i in range(int(n)):
            box_print("{}|{}|{}".format(self.spinWheel(),self.spinWheel(),self.spinWheel()),n_unicode=3)
            sleep(t_b)
            clear_output(wait=True)

    def EV_adjuster(self,JP,base_ev = -1e-3,JP0=1200,lower_bound = -0.6):
        "Adjust the expected value such that the deal is worser as the JP is empied. It converge to 0 as JP increase to infinity"
        Ratio = JP/JP0
        if Ratio < 1:
            return lower_bound*(1-Ratio)**2
        else:
            return (1-(Ratio - base_ev )/Ratio)

    def get_prize(self, target_ev = -1e-3 ,plot = False ,tol = 1e-2,Jackpot = 1200,ignore_Jp = True,TICKET_PRICES = list(np.linspace(0.5,30,2000))):
            "Return the Prize for Ev to hit target_ev per dollar"
            #Intialize
            if ignore_Jp:
                p_jp = 0
            else:
                p_jp = (1/24)**3
            #Part 1:Examine the Game
            E = 0
            EV = []
            DF = []
            for comb,pr,p in zip(self.COMBINATION,self.PROBABILITY,self.PRIZES):
                L1 = np.array([[comb,pr,p]])
                E += p*pr
            #Part 2:Calculation
            for ticket_prize in TICKET_PRICES:
                p_lose = 1-sum(self.PROBABILITY)- p_jp #P_JackPot
                ev = p_lose*-ticket_prize  + p_jp*Jackpot + E #Prize form other rewards
                ev_dollar = ev/ticket_prize
                EV.append(ev_dollar)
                DF.append(abs(ev_dollar - target_ev))
                if abs(ev_dollar - target_ev) < tol:
                    return ticket_prize
            #If fail to calculate
            print("fail to calculate")
            if plot:
                plt.plot(TICKET_PRICES,EV, label = "Ev per dollar")
                plt.plot(TICKET_PRICES,target_ev*np.ones(len(TICKET_PRICES)),label ="Target EV")
                plt.plot(TICKET_PRICES,DF,label ="Difference")
                plt.xlabel("Ticket Prices")
                plt.legend()
                plt.ylabel("EV/$")
                plt.title("EV Per dollar")
            if target_ev < -0.5:
                return 30
            else:
                return 2
    def sanity_check(self):
        print("⚠️⚠️⚠️WARNING: The lower the JACKPOINT Remains, the Pricier the ticket will be!⚠️⚠️⚠️\n")
        JP = list(np.linspace(1,2000,100))
        EV_adjusted = []
        TICKET_PRICES = []
        for jp in JP:
            ev = self.EV_adjuster(jp)
            EV_adjusted.append(ev)
            TICKET_PRICES.append(self.get_prize(ev))
        plt.plot(JP,EV_adjusted,label = "Adjusted Ev per dollar")
        plt.xlabel("JackPoint")
        plt.ylabel("EV per dollar/Ticket_Prize")
        plt.plot(JP,TICKET_PRICES,label = "Ticket Price")
        plt.plot(self.balance,self.get_prize(self.EV_adjuster(self.balance)),"o",label = "Current Price")
        plt.title("Ticket Price Trend")
        plt.legend()
        plt.show()

    def reward_table(self):
        Jackpot = self.balance
        L = np.array([["🎯🎯🎯","💲"+str(Jackpot),(1/24)**3]])
        E = 0
        n = 1
        for comb,pr,p in zip(self.COMBINATION,self.PROBABILITY,self.PRIZES):
            n += 1
            L1 = np.array([[comb,"🎫❎"+str(p)+"💲",pr]])
            L = np.concatenate((L,L1),axis = 0)
            E += p*pr
        columns = ["Combination","Return per 🎫 ","Probability"]
        index = [f'Comb{num}' for num in range(n)]
        df = pd.DataFrame(L, columns=columns, index=index)
        return(df)

    def play(self,INIT_stake = 50):
        try:
            self.stake = float(INIT_stake)
        except:
            self.stake = float(input("Please Enter Number ONLY\n"))
        print(self.Rules)
        display(self.reward_table().head(100))
        self.sanity_check()
        print("Current Ticket Prize:💲{}/🎫".format(self.ticket_price))
        playQuestion = self.askPlayer()
        while(self.stake > 0 and playQuestion == True):
            self.ticket_ceiling = round(self.balance/20)
            print("Maximum Ticket For sale:",self.ticket_ceiling)
            not_pass = True
            self.num_ticket = 0
            while not_pass: 
                try:
                    if self.num_ticket == 0:
                        self.num_ticket = int(input("How many tickets 🎫 would you like to purchase?\n"))
                    if self.num_ticket * self.ticket_price > self.stake:
                        print("Sorry, you can't afford this!")
                        int("FAIL")
                    elif self.num_ticket > self.ticket_ceiling:
                        print("Sorry, you can purchase no more than {} tickets".format(self.ticket_ceiling))
                        int("FAIL")
                    else:
                        not_pass = False
                except:
                    self.num_ticket = int(input("Please enter INTEGETS! e.g: [3]\n"))

            self.firstWheel = self.spinWheel()
            self.secondWheel = self.spinWheel()
            self.thirdWheel = self.spinWheel()
            self.printScore()
            print("Your Bet was:"+"🎫"*self.num_ticket)
            ev_new = self.EV_adjuster(self.balance)
            new_prize = self.get_prize(target_ev = ev_new) #Update the prize every round
            if new_prize != None:
                self.ticket_price = new_prize
            print("Current Ticket Prize:💲{}/🎫".format(self.ticket_price))
            playQuestion = self.askPlayer()
            clear_output(wait=True)
        return self.stake
            

    def askPlayer(self):
        '''
        Asks the player if he wants to play again.
        expecting from the user to answer with yes, y, no or n
        No case sensitivity in the answer. yes, YeS, y, y, nO . . . all works
        '''
        while(True):
            os.system('cls' if os.name == 'nt' else 'clear')
            if (self.balance <=1):
                print ("🤩🤩🤩WOW,Impressive.You have Emptied the Slot Machine.\n JackPoint is now reset.")
                self.balance = 1200

            print ("The Jackpot is currently: 💲" + str(self.balance) + ".")
            answer = input("Enter【Y】to Play; 【C】 to check your money; 【R】 to see the rules; 【Q】 to QUIT\n")
            answer = answer.lower()
            if(answer == "Y" or answer == "y"):
                return True
            elif(answer == "Q" or answer == "q"):
                print("You ended the game with 💲" + str(self.stake) + " in your hand. Great job!")
                return False
            elif(answer == "C" or answer == "c"):
                print ("You currently have 💲" + str(self.stake) + ".")
            elif(answer == "R" or answer == "r"):
                print(self.Rules)
                display(self.reward_table().head(100))
                self.sanity_check()
            else:
                print("Whoops! Didn't get that.")

    def spinWheel(self):
        '''
        returns a random item from the wheel
        '''
        ITEMS = ["🍑", "🍒", "🍇", "🔔", "👑", "🎯"]
        WEIGHTS = [6/24,6/24,6/24,3/24,1/24,1/24]
        return random.choices(ITEMS,weights = WEIGHTS)[0]


    def printScore(self):
        '''
        prints the current score, aks, the PRIZE
        '''
        self.shine()
        if((self.firstWheel == "🍑") and (self.secondWheel != "🍑")):
            print("❎"*2)
            win = 2*self.num_ticket
            self.balance = self.balance - win
        elif((self.firstWheel == "🍑") and (self.secondWheel == "🍑") and (self.thirdWheel != "🍑")):
            win = 5*self.num_ticket
            print("❎"*5)
            self.balance = self.balance - win
        elif((self.firstWheel == "🍑") and (self.secondWheel == "🍑") and (self.thirdWheel == "🍑")):
            win = 7*self.num_ticket
            print("❎"*7)
            self.balance = self.balance - win
        elif((self.firstWheel == "🍒") and (self.secondWheel == "🍒") and ((self.thirdWheel == "🍒") or (self.thirdWheel == "👑"))):
            win = 10*self.num_ticket
            print("❎"*10)
            self.balance = self.balance - win
        elif((self.firstWheel == "🍇") and (self.secondWheel == "🍇") and ((self.thirdWheel == "🍇") or (self.thirdWheel == "👑"))):
            win = 14*self.num_ticket
            print("❎"*14)
            self.balance = self.balance - win
        elif((self.firstWheel == "🔔") and (self.secondWheel == "🔔") and ((self.thirdWheel == "🔔") or (self.thirdWheel == "👑"))):
            win = 20*self.num_ticket
            print("❎"*20)
            self.balance = self.balance - win
        elif((self.firstWheel == "👑") and (self.secondWheel == "👑") and (self.thirdWheel == "👑")):
            win = 250*self.num_ticket
            print("❎"*250)
            self.balance = self.balance - win
        elif((self.firstWheel == "🎯") and (self.secondWheel == "🎯") and (self.thirdWheel == "🎯")):
            display(Loaded_pics["JackPot"])
            win = self.balance
            self.balance = self.balance - win
        else:
            win = -self.num_ticket
            self.balance = self.balance + self.num_ticket

        self.stake += win
        if win == self.balance:
            print ("You won the JACKPOT!!")
        elif (win > 0):
            box_print(self.firstWheel + '|' + self.secondWheel + '|' + self.thirdWheel,n_unicode = 3)
            print("🎉🎉🎉You win 💲" + str(win)) 
            os.system('cls' if os.name == 'nt' else 'clear')
        else:
            box_print(self.firstWheel + '|' + self.secondWheel + '|' + self.thirdWheel,n_unicode = 3)
            print("😢😢😢You lose")
            os.system('cls' if os.name == 'nt' else 'clear')

# Play Ground

In [ ]:
test = slot_machine()


In [ ]:
test.play(3653.0)